# Battleships

## Planning

- standard version & extended version (if we have time)
    - standard
        - normal rules
        - 2 human players
        - basic display (jupyter output)
    - extended
        - graphics display (turtle?)
        - hard mode ("Salvo")
        - computer player
            - random guess
            - if last guess == hit, random guess within closer range...
        - different size board
        - different size ships(?)


Standard:

- board
    - 10x10
    - 4 boards (player_1_sea, player_1_radar, player_2_sea, player_2_radar)

- ships
    - number/size has to make sense for the board dimensions
    - standard:
        - Carrier (5)
        - Battelship (4)
        - Cruiser (3)
        - Submarine (3)
        - Destroyer (2)

- turn
    - guess one coordinate on board
        - outputs result (miss / hit + ship if hit)
            - sunk if ship gone
        - displays new board (updated pegs)

- display
    - can only see your board

Functions:

- display_board
    - input lists
    - output (print) sea and radar

- player_names
    - input string

- place_ships
    - choose ship
    - choose starting coordinate
    - choose direction (udlr)
    - calls check_space_free (iterates for all coords in selection)
    - updates ship variables

- update board

- check_space_free

- choose_first_player
    - random (0, 1)

- fire_shot
    - choose location to fire at
    - update board/list

- check_hit
    - returns result of shot
    - update ship count

- check_ship_status
    - checks if 

- save_shot
    - remove available firing locations
    - decide whether can play in same square again

- ready_for_turn
    - when turn ended, hide board
    - ask if ready
    - when ready, show other player board

- winning_condition
    - ship count == 0

- replay

Game Logic

- initiate empty board
- ask for player names
- decide who goes first

- player 1 places ships
    - display_board per ship placed
    - ready_for_turn
- player 2 places ships
    - display_board per ship placed
    - ready_for_turn

- while not winning_condition:
    - player 1 fire_shot
        - display board
        - (update board)
        - check_hit
        - check_ship_status
        - check winning_condition
        - display board
        - ready_for_turn
    - player 2 fire_shot (same logic as player 1)

- display result
    - display both boards (?)

- ask for replay

In [1]:
print("\U0001F6E5")
print("\U000026F4")
print("\U0001F4A5")

🛥
⛴
💥


## Functions

In [2]:
# Library imports
import random
from getpass import getpass

import IPython
from IPython.display import display, clear_output

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout

In [3]:
def player_names():
    """Asks for the names of the players and returns both in a tuple."""
    player_1 = input("Player name: ")
    player_2 = input("Player name: ")
    return player_1, player_2

def choose_first_player(player_1_name, player_2_name):
    """Randomly selects which player has the first turn. Returns one of the two player names that were 
    passed to the function."""
    if random.randint(0,1) == 0:
        return player_1_name
    else:
        return player_2_name

def display_boards(player_name, radar_lst, sea_lst):
    """Prints the radar_lst and sea_lst lists in the format of Battleships boards, with the letters list
    denoting the rows."""
        
    print(player_name, "\n")
    print("Radar\n")
    print("     1   2   3   4   5   6   7   8   9   10")
    for index, i in enumerate(radar_lst):
        if index == 0:
            pass
        elif index % 10 == 1:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 0:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")
            
    print("\nSea\n")
    print("     1   2   3   4   5   6   7   8   9   10")
    for index, i in enumerate(sea_lst):
        if index == 0:
            pass
        elif index % 10 == 1:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 0:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")

In [4]:
def update_board_placement(sea_lst, boat_coords, symbol):
    """Updates the board after the placement of a boat, taking a list of boat_coords as input."""
    for coord in boat_coords:
        sea_lst[coord] = symbol
        
def update_board_combat(radar_lst, sea_lst, position, flag):
    """Taking the position and flag results from a turn, this function updates the radar_lst and 
    sea_lst lists that make up the board."""
    if flag:
        radar_lst[position] = radar_hit
        sea_lst[position] = sea_hit
    else:
        radar_lst[position] = radar_miss
        sea_lst[position] = sea_miss
        
def update_scoreboard():
    """Updates the scoreboard."""
    with scoreboard:
        clear_output()
        print("Scoreboard: \n")
        print(f"{player_1_name} has sunk:" + "\n"*(5-len(player_2_sunk_boats)))
        if player_2_sunk_boats:
            display(player_2_sunk_boats)
        print(f"{player_2_name} has sunk:")
        if player_1_sunk_boats:
            display(player_1_sunk_boats)

In [5]:
def enforce_player_coord(coord):
    """Ensures a player coordinate input is in the valid 'A1' - 'J10' format. Returns coord once in the correct format."""
    # Ensure second item in coord is a number
    try:
        int(coord[1])
    except (ValueError, IndexError):
        coord = input("Enter coordinates (in the format 'A1'): ").upper()

    # Ensure coord is 2-3 characters long (and ends in '10') if 3 characters, starts with a letter in letters and
    # ends in a number 1-10
    while len(coord) > 3 or len(coord) < 2 or \
    (len(coord) == 3 and coord[1:3] != "10") or \
    coord[0] not in letters or int(coord[1]) not in range(1, 10):
        with text_update:
            print("Coordinates must be in format 'LetterNumber' (e.g. 'A1').")
        coord = input("Enter coordinates (in the format 'A1'): ").upper()
     
    return coord.upper()

def coord_string_to_digit(coord):
    return letters.index(coord[0]) * 10 + int(coord[1:])

In [6]:
def place_boats(player_name, radar_lst, sea_lst, player_boat_coords):
    """Prompts user to place boats in descending order of length, by entering starting co-ordinate and direction.
    Calls check_space_free() to ensure boat placement is valid - if not, prompts user to place boat elsewhere.
    Saves placed boats in player_boats list."""
    
    for boat, data in boat_types.items():
        length, symbol = data
        coords = ""
        while not coords:            
#             if not first_pass:
#                 with text_update:
#                     print(txt)
            start_coord = coord_string_to_digit(enforce_player_coord(getpass(f"Enter start coordinate for {boat}: ").upper()))
            direction = ""
            first_dir = True
            while direction not in ["U","D","L","R"]:
                if not first_dir:
                    with text_update:
                        print("Direction must be one of 'U', 'D', 'L' or 'R'.")
                direction = getpass("Choose direction (U/D/L/R): ").upper()
                first_dir = False
            coords = check_space_free(sea_lst, start_coord, direction, length)
#             first_pass = False
#         with text_update:
#             print(txt)
        player_boat_coords.append(coords)
        update_board_placement(sea_lst, coords, symbol)

        with boards:
            clear_output()
            display(display_boards(player_name, radar_lst, sea_lst))

In [7]:
def check_space_free(sea_lst, start_coord, direction, length):
    """Checks if all required spaces for boat placement are free and in range.
    Returns tuple (coordinates, "Ship placed") if valid, otherwise returns (False, [error message])."""
    
    # use length of boat and direction to find all coords it will cover
    # if any of those coords are not sea empty, return False
        # also handle out of range issues
    # else return coords

    if sea_lst[start_coord] != sea_empty:
        with text_update:
            print("Sea already occupied here")
        return False
    
    boat_coords = [start_coord]
    row = start_coord // 10
    if start_coord % 10 == 0:
        row -= 1
                
    if direction in ["L", "R"]:
        d = 1
        if direction == "L":
            d = -1
        for i in range(1, length):
            next_coord = start_coord + (i * d)
            if (next_coord - 1) // 10 != row:
                with text_update:
                    print("Ship outside board")
                return False
            elif sea_lst[next_coord] != sea_empty:
                with text_update:
                    print("Sea already occupied here")
                return False
            else:
                boat_coords.append(next_coord)
        with text_update:
            print("Ship placed")
        return boat_coords
    
    elif direction in ["U", "D"]:
        d = 1
        if direction == "U":
            d = -1
        for i in range(1, length):
            next_coord = start_coord + (i * 10 * d)
            if next_coord not in range(1, 101):
                with text_update:
                    print("Ship outside board")
                return False
            elif sea_lst[next_coord] != sea_empty:
                with text_update:
                    print("Sea already occupied here")
                return False
            else:
                boat_coords.append(next_coord)
        with text_update:
            print("Ship placed")
        return boat_coords
    
    else:
        with text_update:
            print("Direction error")
        return False

In [8]:
def fire_shot(player_guesses):
    """Calls enforce_player_coord() to prompt player to enter a guess and enforce validity.
    If coordinates have been guessed by the player already, asks for another guess.
    If the guess is valid, adds the guess to player_guesses and returns the 
    list index ('position') of the player guess."""
    
    guess = ""
    guess = enforce_player_coord(guess)
    
    # Checks if guess has already been guessed in a previous turn
    while guess in player_guesses:
        with text_update:
            print("Coordinates already guessed!")
        guess = input("Enter coordinates (in the format 'A1'): ").upper() 
    
    player_guesses.append(guess) # adds guess to guess history
    position = coord_string_to_digit(guess)
    return position

In [64]:
def check_hit(sea_lst, position):
    """Takes player guess position and evaluates if it matched an opponent boat. Updates text_update and returns bool."""
    if sea_lst[position] in sea_boats:
        with text_update:
            print("Hit!")
        return True
    else:
        with text_update:
            print("Miss!")
        return False

In [11]:
def check_ship_status(sea_lst, player_boat_coords, player_sunk_boats):
    """Checks which ships are destroyed and updates ship count."""
    for index, boat in enumerate(player_boat_coords):
        if all([sea_lst[boat[i]] == sea_hit for i in range(len(boat))]):
            boat_type = list(boat_types.keys())[index]
            if boat_type not in player_sunk_boats:
                with text_update:
                    print(f"{boat_type} sunk!")
            player_sunk_boats.add(boat_type)

In [12]:
def winning_condition():
    """Updates global winner variable and returns True if either player has 5 ships sunk."""
    global winner
    if len(player_1_sunk_boats) == 5:
        winner = player_2_name
        return True
    elif len(player_2_sunk_boats) == 5:
        winner = player_1_name
        return True
    else:
        return False

In [13]:
def new_turn(name):
    """When turn ended (i.e. function is called), hides the previous player's board and asks next player if they are ready.
    Once second player is ready, sets turn to their go and displays their board."""
    
    global turn
    boards.clear_output()
    with text_update:
        clear_output()
        print("Text updates:\n")
    ready = input(f"{name}, enter Y to continue: ").upper()
    if ready == "Y":
        if name == player_1_name:
            with boards:
                display(display_boards(name, radar_lst_p1, sea_lst_p1))
        else:
            with boards:
                display(display_boards(name, radar_lst_p2, sea_lst_p2))
    turn = name
    with text_update:
        print(f"{name} - your go!")

In [14]:
def final_result(winner):
    """Displays both boards at the end of the game."""
    clear_output()
    print(f"{winner} wins!\n")
    if winner == player_1_name:
        display(display_boards(player_1_name, radar_lst_p1, sea_lst_p1))
    else:
        display(display_boards(player_2_name, radar_lst_p2, sea_lst_p2))

In [66]:
def test_variables():
    """Returns test variables for two players in the order:
    - test_radar
    - test_sea
    - test_boat_coords,
    - test_sunk_boats
    - test_guesses"""
    
    test_radar = [radar_empty]*101
    test_sea = [sea_empty] * 101
    test_radar2 = [radar_empty]*101
    test_sea2 = [sea_empty] * 101
    
    test_boat_coords = [[1,2,3,4,5], [11,12,13,14], [21,22,23], [31,32,33], [41,42]]
    test_boat_coords2 = [[100,90,80,70,60], [99,89,79,69], [98,88,78], [97,87,77], [96,86]]
    
    # Set player1_sea and player2_radar to all boats coords hit except last for each boat, and misses for opposite boards
    for boat, coords in enumerate(test_boat_coords):
        for i in range(len(coords)-1):
            test_sea[coords[i]] = sea_hit
            test_radar2[coords[i]] = radar_hit
            test_sea2[coords[i]] = sea_miss
            test_radar[coords[i]] = radar_miss
        test_sea[coords[len(coords)-1]] = sea_boats[boat]
    
    for boat, coords in enumerate(test_boat_coords2):
        for i in coords:
            test_sea2[i] = sea_boats[boat]
    
    
    test_sunk_boats = set()
    test_guesses = []
    test_sunk_boats2 = set()
    test_guesses2 = []
    
    return test_radar, test_sea, test_boat_coords, test_sunk_boats, test_guesses, \
            test_radar2, test_sea2, test_boat_coords2, test_sunk_boats2, test_guesses2

In [58]:
a, b, c, d, e, v, w, x, y, z = test_variables()

display_boards("test", a, b)
# display_boards("test2", v, w)

test 

Radar

     1   2   3   4   5   6   7   8   9   10
A: | o | o | o | o |   |   |   |   |   |   |
B: | o | o | o |   |   |   |   |   |   |   |
C: | o | o |   |   |   |   |   |   |   |   |
D: | o | o |   |   |   |   |   |   |   |   |
E: | o |   |   |   |   |   |   |   |   |   |
F: |   |   |   |   |   |   |   |   |   |   |
G: |   |   |   |   |   |   |   |   |   |   |
H: |   |   |   |   |   |   |   |   |   |   |
I: |   |   |   |   |   |   |   |   |   |   |
J: |   |   |   |   |   |   |   |   |   |   |

Sea

     1   2   3   4   5   6   7   8   9   10
A: |~x~|~x~|~x~|~x~|~A~|~~~|~~~|~~~|~~~|~~~|
B: |~x~|~x~|~x~|~B~|~~~|~~~|~~~|~~~|~~~|~~~|
C: |~x~|~x~|~S~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
D: |~x~|~x~|~C~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
E: |~x~|~D~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
F: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
G: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
H: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
I: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
J: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~

In [67]:
# Display containers
text_update = widgets.Output()
boards = widgets.Output()
scoreboard = widgets.Output()

app =  AppLayout(header=None,
                 left_sidebar=text_update,
                 center=boards,
                 right_sidebar=scoreboard,
                 footer=None,
#                  pane_heights=[1, 20, 1]
                )

print("Welcome to Battleships!\n")
display(app)

# Reset boards and define initial variables
letters = ["A","B","C","D","E","F","G","H","I","J"]

# Radar squares
radar_empty = "   "
radar_boats = [" D ", " C ", " S ", " B ", " A "]
radar_hit = " x "
radar_miss = " o "

# Sea squares
sea_empty = "~~~"
sea_boats = ["~A~", "~B~", "~S~", "~C~", "~D~"]
sea_hit = "~x~"
sea_miss = "~o~"

# Boat types
boat_types = {"Aircraft carrier (5)" : (5, "~A~"), 
              "Battleship (4)" : (4, "~B~"), 
              "Submarine (3)" : (3, "~S~"), 
              "Cruiser (3)" : (3, "~C~"), 
              "Destroyer (2)" : (2, "~D~")}

# Create empty grids
sea_lst_p1 = [sea_empty] * 101  
radar_lst_p1 = [radar_empty] * 101
sea_lst_p2 = [sea_empty] * 101  
radar_lst_p2 = [radar_empty] * 101

# Set of player boats that have been sunk
player_1_sunk_boats = set()
player_2_sunk_boats = set()

# Lists of player boat locations
player_1_boat_coords = []
player_2_boat_coords = []

# Lists of past guesses
player_1_guesses = []
player_2_guesses = []

replay_dropdown = widgets.Dropdown(
                    options=['','Yes', 'No'],
                    value='',
                    description='Replay? ',
                    disabled=False,
                    )

### Game starts ###
winner = False

# Ask for player names
player_1_name, player_2_name = player_names()

with text_update:
    print("Text updates:\n")
with scoreboard:
    print("Scoreboard:\n")
    print(f"{player_1_name} has sunk:" + "\n"*5)
    print(f"{player_2_name} has sunk:")


# Allow endgame testing
if player_1_name.lower() == "test":
    radar_lst_p1, sea_lst_p1, player_1_boat_coords, player_1_sunk_boats, player_2_guesses, \
    radar_lst_p2, sea_lst_p2, player_2_boat_coords, player_2_sunk_boats, player_2_guesses = test_variables()
    turn == player_2_name
    with boards:
        display_boards(player_2_name, radar_lst_p2, sea_lst_p2)
    
else:
    # Determine which player starts
    turn = choose_first_player(player_1_name, player_2_name)    
    with text_update:
        print(f"\n{turn} starts!\n")

    # Players placing the boats
    if turn == player_1_name:
        # Player 1 places ships
        with boards:
            display_boards(player_1_name, radar_lst_p1, sea_lst_p1)
        place_boats(player_1_name, radar_lst_p1, sea_lst_p1, player_1_boat_coords)
        new_turn(player_2_name)
        # Player 2 places ships
        place_boats(player_2_name, radar_lst_p2, sea_lst_p2, player_2_boat_coords)
        new_turn(player_1_name)
    else:
        # Player 2 places ships
        with boards:
            display_boards(player_2_name, radar_lst_p2, sea_lst_p2)
        place_boats(player_2_name, radar_lst_p2, sea_lst_p2, player_2_boat_coords)
        new_turn(player_1_name)
        # Player 1 places ships
        place_boats(player_1_name, radar_lst_p1, sea_lst_p1, player_1_boat_coords)
        new_turn(player_2_name)

# Taking turns to fire
while not winning_condition():
    if turn == player_1_name:
    # Player 1 turn to fire
        # Get index position of player guess
        position = fire_shot(player_1_guesses)
        # Check if player guess was a hit
        flag = check_hit(sea_lst_p2, position)
        # Update Player 1 radar and Player 2 sea with result of shot
        update_board_combat(radar_lst_p1, sea_lst_p2, position, flag)
        # Check if any of Player 2's ships sunk, update sunk_ships list 
        check_ship_status(sea_lst_p2, player_2_boat_coords, player_2_sunk_boats)
        update_scoreboard()
        # Check if winning condition fulfilled
        if winning_condition():
            final_result(winner)
            break
        # Display updated Player 1 boards
        with boards:
            clear_output()
            display_boards(player_1_name, radar_lst_p1, sea_lst_p1)

        # Call new_turn() to initiate change of turn
        ready = ""
        while ready != "Y":
            ready = input(f" Ready to pass to {player_2_name}? ").upper()
        new_turn(player_2_name)

    else:
    # Player 2 turn to fire
        # Get index position of player guess
        position = fire_shot(player_2_guesses)
        # Check if player guess was a hit
        flag = check_hit(sea_lst_p1, position)
        # Update Player 2 radar and Player 1 sea with result of shot
        update_board_combat(radar_lst_p2, sea_lst_p1, position, flag)
        # Check if any of Player 1's ships sunk, update sunk_ships list 
        check_ship_status(sea_lst_p1, player_1_boat_coords, player_1_sunk_boats)
        update_scoreboard()
        # Check if winning condition fulfilled
        if winning_condition():
            final_result(winner)
            break
        # Display updated Player 2 boards
        with boards:
            clear_output()
            display_boards(player_2_name, radar_lst_p2, sea_lst_p2)

        # Call new_turn() to initiate change of turn
        ready = ""
        while ready != "Y":
            ready = input(f" Ready to pass to {player_1_name}? ").upper()
        new_turn(player_1_name)

        
# display(replay_dropdown)

Welcome to Battleships!



AppLayout(children=(Output(layout=Layout(grid_area='left-sidebar')), Output(layout=Layout(grid_area='right-sid…

Player name: test
Player name: James
Enter coordinates (in the format 'A1'): a5
 Ready to pass to test? y
test, enter Y to continue: y
Enter coordinates (in the format 'A1'): a7
 Ready to pass to James? y
James, enter Y to continue: y
Enter coordinates (in the format 'A1'): D3


KeyboardInterrupt: Interrupted by user

In [68]:
# manage(clear) text output
    # user inputs in comboboxes
    # comboboxes in footer?
    # enforce_coord - check whether to use getpass or input?

# replay function    

# None under grids?




In [380]:
def replay():
    """Asks players if they want to play again. If 'Yes', restarts game. If 'No', exits program."""
    if change['new'] == "Yes":
        print("True")
        return True
    else:
        print("False")
        return False

In [383]:
output3 = widgets.Output()

def replay_change(change):
    with output3:
        print(change['new'])

display(replay_dropdown, output3)

replay_dropdown.observe(replay_change, names='value')

Dropdown(description='Replay? ', options=('', 'Yes', 'No'), value='')

Output()

In [384]:
int_range = widgets.IntSlider()
output2 = widgets.Output()

display(int_range, output2)

def on_value_change(change):
    with output2:
        print(change['new'])

int_range.observe(on_value_change, names='value')

IntSlider(value=0)

Output()

In [289]:
tester = widgets.Output()

with tester:
    input("\rTest: ")
    
with tester:
    


Test: Help


In [250]:
print("Important")
input("This is a test: ")
clear_output(wait=True)
input("And again: ")

And again: Is it still there?


'Is it still there?'

In [224]:
import time
print('This is important info!')
for i in range(100):
    print("\r"+'Processing BIG data file {}'.format(i),end="")
    time.sleep(0.1)
    if i == 50:
        print("\r"+'Something bad happened on run {}.  This needs to be visible at the end!'.format(i))
print("\r"+'Done.')inpu

This is important info!
Something bad happened on run 50.  This needs to be visible at the end!
Done.ssing BIG data file 99


In [232]:
print("XXXX", end="\r")
print("YYYY")

YYYY


In [246]:
from IPython.utils import io
with io.capture_output() as captured:
    pass